In [4]:
import pandas as pd 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [5]:
#df = pd.read_csv("../Dataset/spotify_dataset.csv") #Chargement du Dataset original

# Tranformation des Dataset

### Spotify_dataset_transformed.csv

In [6]:
df = pd.read_csv("../Dataset/spotify_dataset.csv") # Chargement du Dataset original

def length_to_seconds(length):
    if pd.notna(length):
        parts = str(length).split(":")
        if len(parts) == 2:
            minutes = int(parts[0])
            seconds = int(parts[1])
            return minutes * 60 + seconds
    return 0

def get_most_popular_genre(genres_str):
    if pd.isna(genres_str):
        return None
    genres = [g.strip() for g in genres_str.split(',')]
    for top_genre in top_genres:
        if top_genre in genres:
            return top_genre
    return None  # Aucun des top genres n'est présent
    
# Genre transformation
top_genres = ['hip hop', 'pop', 'rock', 'trap', 'alternative rock', 'rap', 'pop rock', 'electronic']
df_copy = df.copy()


#Transformation text/Length/Genre

df_copy['text'] = df_copy['text'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0) #A Réécrire
df_copy['Length'] = df_copy['Length'].apply(length_to_seconds) #Conversion du Length en secondes
df_copy['Main_genre'] = df_copy['Genre'].apply(get_most_popular_genre)

nb_none = df_copy['Main_genre'].isna().sum()
df_transformed = df_copy.dropna(subset=['Main_genre']).copy()
print(f"Nombre de lignes avant suppression : {len(df_copy)}")
print(f"Nombre de lignes après suppression : {len(df_transformed)}")

#Mapping by using Main_genre_mapping
main_genre_mapping = {genre: idx for idx, genre in enumerate(top_genres)}
df_transformed["Main_genre_index"] = df_transformed["Main_genre"].map(main_genre_mapping)

df_transformed.to_csv("../Dataset/spotify_dataset_transformed.csv", index=False)
df_transformed.head(5)[["song","Artist(s)",'Main_genre',"Main_genre_index","Genre"]]

Nombre de lignes avant suppression : 498052
Nombre de lignes après suppression : 398157


,song,Artist(s),Main_genre,Main_genre_index,Genre
0,Even When the Waters Cold,!!!,hip hop,0,hip hop
1,One Girl / One Boy,!!!,hip hop,0,hip hop
2,Pardon My Freedom,!!!,hip hop,0,hip hop
3,Ooo,!!!,hip hop,0,hip hop
4,Freedom 15,!!!,hip hop,0,hip hop


### Spotify_dataset_10k.csv

In [7]:
df = pd.read_csv("../Dataset/spotify_dataset_transformed.csv")
df_10k = df.sample(n=10000, random_state=42)
df_10k.to_csv("../Dataset/spotify_dataset_10k.csv", index=False)

### Spotify_dataset_30k.csv

In [8]:
import pandas as pd
import numpy as np

spotify_df_2 = pd.read_csv("../Dataset/spotify_dataset_transformed.csv")
# Paramètres
target_size = 40000
genre_fraction = 0.125  # 12.5% pour chaque genre
pop_frac = {'popular': 0.4, 'non_popular': 0.6}

# Dictionnaire final pour stocker les échantillons
samples = []

# On boucle sur chaque genre
for genre in top_genres:
    genre_df = spotify_df_2[spotify_df_2["Main_genre"] == genre]
    
    # On sépare les populaires et non populaires
    popular = genre_df[genre_df["Popularity"] >= 70]
    non_popular = genre_df[genre_df["Popularity"] < 70]
    
    # Nombre total à prendre pour ce genre
    genre_target = int(target_size * genre_fraction)
    
    # 40% populaires, 60% non populaires
    pop_target = int(genre_target * pop_frac['popular'])
    non_pop_target = genre_target - pop_target  # ou int(genre_target * pop_frac['non_popular'])

    # On prend un échantillon (ou tout si moins)
    pop_sample = popular.sample(n=min(pop_target, len(popular)), random_state=42)
    non_pop_sample = non_popular.sample(n=min(non_pop_target, len(non_popular)), random_state=42)
    
    # Ajout au résultat final
    samples.extend([pop_sample, non_pop_sample])

# Concaténer tous les échantillons

spotify_df_reduced = pd.concat(samples, ignore_index=True)

# Vérification des tailles
print("Taille finale du dataset réduit :", len(spotify_df_reduced))
print(spotify_df_reduced["Main_genre"].value_counts())
print("Popularité >= 70 :", len(spotify_df_reduced[spotify_df_reduced["Popularity"] >= 70]))
print("Popularité < 70 :", len(spotify_df_reduced[spotify_df_reduced["Popularity"] < 70]))
print(top_genres)
spotify_df_reduced.to_csv("../Dataset/spotify_dataset_30k.csv", index=False)


Taille finale du dataset réduit : 29233
Main_genre
hip hop             5000
pop                 5000
rock                5000
alternative rock    3508
trap                3263
pop rock            3211
electronic          3160
rap                 1091
Name: count, dtype: int64
Popularité >= 70 : 7154
Popularité < 70 : 22079
['hip hop', 'pop', 'rock', 'trap', 'alternative rock', 'rap', 'pop rock', 'electronic']


### Spotify_dataset_popularity_balanced.csv

In [9]:
df = pd.read_csv("../Dataset/spotify_dataset_transformed.csv")

# Séparer les morceaux populaires et non populaires
popular = df[df["Popularity"] > 70]
non_popular = df[df["Popularity"] <= 70]

# Objectif : 40% de morceaux populaires, 60% de non populaires
nb_popular = 8000
nb_non_popular = int((0.6 * nb_popular) / 0.4)  # soit 15000

# Échantillonnage aléatoire
popular_sampled = popular.sample(n=nb_popular, random_state=42)
non_popular_sampled = non_popular.sample(n=nb_non_popular, random_state=42)

# Fusionner et mélanger
balanced_df = pd.concat([popular_sampled, non_popular_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Vérification
l1 = len(balanced_df[balanced_df["Popularity"] > 70])
l2 = len(balanced_df[balanced_df["Popularity"] <= 70])
print("Total :", len(balanced_df))
print("Popularité > 70 :", l1)
print("Popularité ≤ 70 :", l2)
print("Proportions :", round(l1 / (l1 + l2), 2), "/", round(l2 / (l1 + l2), 2))

balanced_df.to_csv("../Dataset/spotify_dataset_popularity_balanced.csv", index=False)
balanced_df.head(10)[["song","Popularity"]]

Total : 20000
Popularité > 70 : 8000
Popularité ≤ 70 : 12000
Proportions : 0.4 / 0.6


,song,Popularity
0,No. 9,62
1,‎you me,85
2,I Love You,7
3,J-Bieber Rap,79
4,2017 Mashup,25
5,Hosanna,15
6,TI.PO.TA - Moonlight Avenue,50
7,Team,84
8,ORIGAMI,9
9,Victory or Death,17


### Spotify_dataset_genre_balanced.csv

In [10]:
spotify_df = pd.read_csv("../Dataset/spotify_dataset_30k.csv")

genre_counts = spotify_df['Main_genre'].value_counts()
total_desired = 12000  # taille cible

# On garde un déséquilibre léger basé sur les proportions initiales
# Mais on "lisse" vers l'équilibre : on redistribue les proportions
base_proportions = genre_counts / genre_counts.sum()

# On utilise un facteur de lissage : on évite qu'une classe dépasse trop les autres
# Ici on "lisse" à mi-chemin entre l'équilibre pur et les proportions réelles
equal_proportion = 1 / len(genre_counts)
smooth_proportions = 0.5 * base_proportions + 0.5 * equal_proportion

# On convertit ça en effectif pour chaque genre
target_counts = (smooth_proportions * total_desired).astype(int)

# Affichage des cibles par genre
print("Cibles équilibrées (légèrement déséquilibrées) :")
for genre in genre_counts.index:
    print(f"{genre:17} : {target_counts[genre]} morceaux")


balanced_samples = []

for genre in genre_counts.index:
    genre_df = spotify_df_reduced[spotify_df_reduced["Main_genre"] == genre]
    count = target_counts[genre]
    sampled = genre_df.sample(n=min(count, len(genre_df)), random_state=42)
    balanced_samples.append(sampled)

spotify_df_balanced = pd.concat(balanced_samples, ignore_index=True)

print(f"\nTaille finale du dataset équilibré : {len(spotify_df_balanced)}")
print("\nRépartition réelle :")
print(spotify_df_balanced['Main_genre'].value_counts(normalize=True).round(3))

spotify_df_balanced.to_csv("../Dataset/spotify_dataset_class_balanced.csv", index=False)

Cibles équilibrées (légèrement déséquilibrées) :
hip hop           : 1776 morceaux
pop               : 1776 morceaux
rock              : 1776 morceaux
alternative rock  : 1470 morceaux
trap              : 1419 morceaux
pop rock          : 1409 morceaux
electronic        : 1398 morceaux
rap               : 973 morceaux

Taille finale du dataset équilibré : 11997

Répartition réelle :
Main_genre
hip hop             0.148
pop                 0.148
rock                0.148
alternative rock    0.123
trap                0.118
pop rock            0.117
electronic          0.117
rap                 0.081
Name: proportion, dtype: float64
